In [1]:
#pip3 install imageio PyPDF2 wand imgaug opencv-python
#sudo vim /etc/ImageMagick-6/policy.xml
# if wanting to read pdfs change PDF policy to read
# change memory limit to higher like 3GB
import imageio, glob

import numpy
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
import cv2

import zipfile

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, array_to_img

import matplotlib.pyplot as plt

tf.enable_eager_execution()

#read https://stackoverflow.com/questions/52462058/keras-image-preprocessing/52462868

### ProNet Preprocessing Notebook - for FLIC dataset


This notebook is designed to evaluate all of the image preprocessing methods proposed in design document. 

Each cell is designed to be self contained. 

As of current, each cell is not exhaustive. So not all methods have been explored for each package (TODO)

TODO
1. Decide on which preprocessing methods to undertake
2. Create spreadsheet of both undertaken 

Evaluation Process
1. 11 classes, 11 test classes - test examples not uniform, train examples uniform
2. Only 150 frames of video will be used per class. This represents a sparse training apporach that we can use to evaluate
3. Beyond 150 images per class, the rest of the images will be used to generate a test dataset. This means test dataset images have never been seen, but are still similiar however as taken out of sequence.
4. We will train and evaluate our vanilla dataset (train and test) on a regular CNN first, to get a base accuracy.
5. We will then convert vanilla dataset to proNet compatible, train on a single CNN, to get a proNet accuracy

If this is promising, we will stick with this approach, instead of multiple CNN model approach. We wll then go forth with image preprocessing.

If not promising, we will target movie faking, by modifying scenes ALREADY OBSERVED in the training dataset, preprocessed via various means, as test dataset. 



28.12.2019: 
Initial results with 11 classe x 150 images not promising. Simply not enough data.We should cut down to five different movie classes, 400 images each at least. Try again with that and observe results. 
Then make conclusions







In [3]:
#Loading ProNet Data

#!gdown https://drive.google.com/uc?id=1yQrnETzru-ios7TO-Qeitmx_HHCW25W3

#!unzip pronet_new.zip

#Caltech Raw#
#!gdown https://drive.google.com/uc?id=1FUrPfZ9aB4-CwkbmJD9xm-r_AtdQV41u
#!unzip CaltechDataset.zip

#Separated dataset
!gdown https://drive.google.com/uc?id=1LhyI2F0k3PbFo4A6YQ9HEg5pQera2NoF
!unzip CaltechDataset_separated_10.zip


Streaming output truncated to the last 5000 lines.
  inflating: CaltechDataset/Caltech101/grand_piano/image_0063.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0064.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0065.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0066.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0067.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0068.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0069.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0070.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0071.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0072.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0074.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0075.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0076.jpg  
  inflating: CaltechDataset/Caltech101/grand_piano/image_0077.jpg

In [0]:
# Define default directories
base_path = '../content/CaltechDataset/Caltech101'
test_image_path ='../content/CaltechDataset/test'



In [0]:
#Resolution Changing Test - Success 25/12/2019
#SEE HERE https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
#Scale down and scale back up, to generate a loss quality. 
#Hypothesis: necessary, as smaller size would result in problem with CNN input image size

#TODO write up progressively scanning approach to CNN

from google.colab.patches import cv2_imshow
import cv2
def scaleImage(imagepath,percentage):

  img = cv2.imread(imagepath, cv2.IMREAD_UNCHANGED)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  #print('Original Dimensions : ',img.shape)
  width = int(img.shape[1])
  height = int(img.shape[0])
  #scale down percent
  scale_percent = percentage # percent of original size
  newwidth = int(img.shape[1] * scale_percent / 100)
  newheight = int(img.shape[0] * scale_percent / 100)


  dim = (width, height)

  resized =cv2.resize(cv2.resize(img, (newwidth, newheight), interpolation=cv2.INTER_AREA), (width, height), interpolation=cv2.INTER_NEAREST)
  return resized


#print('Resized Dimensions : ',resized.shape)

#cv2_imshow(resized)

In [6]:
# Performing walk to find folderpaths

import os
folderpaths= []
for root, dirs, files in os.walk(base_path, topdown=False):
   #for name in files:
      #print(os.path.join(root, name))
   for name in dirs:
      folderpath = os.path.join(root, name)
      print(folderpath)
      folderpaths.append(folderpath)

../content/CaltechDataset/Caltech101/schooner
../content/CaltechDataset/Caltech101/elephant
../content/CaltechDataset/Caltech101/crocodile
../content/CaltechDataset/Caltech101/pigeon
../content/CaltechDataset/Caltech101/kangaroo
../content/CaltechDataset/Caltech101/pagoda
../content/CaltechDataset/Caltech101/rhino
../content/CaltechDataset/Caltech101/euphonium
../content/CaltechDataset/Caltech101/brain
../content/CaltechDataset/Caltech101/llama
../content/CaltechDataset/Caltech101/sunflower
../content/CaltechDataset/Caltech101/starfish
../content/CaltechDataset/Caltech101/Faces_easy
../content/CaltechDataset/Caltech101/sea_horse
../content/CaltechDataset/Caltech101/crocodile_head
../content/CaltechDataset/Caltech101/garfield
../content/CaltechDataset/Caltech101/binocular
../content/CaltechDataset/Caltech101/dollar_bill
../content/CaltechDataset/Caltech101/panda
../content/CaltechDataset/Caltech101/mayfly
../content/CaltechDataset/Caltech101/hedgehog
../content/CaltechDataset/Caltech101

In [7]:
#Repeat for test directory

testpaths= []
for root, dirs, files in os.walk(test_image_path, topdown=False):
   #for name in files:
      #print(os.path.join(root, name))
   for name in dirs:
      testpath = os.path.join(root, name)
      print(testpath)
      testpaths.append(testpath)

../content/CaltechDataset/test/schooner
../content/CaltechDataset/test/elephant
../content/CaltechDataset/test/crocodile
../content/CaltechDataset/test/pigeon
../content/CaltechDataset/test/kangaroo
../content/CaltechDataset/test/pagoda
../content/CaltechDataset/test/rhino
../content/CaltechDataset/test/euphonium
../content/CaltechDataset/test/brain
../content/CaltechDataset/test/llama
../content/CaltechDataset/test/sunflower
../content/CaltechDataset/test/starfish
../content/CaltechDataset/test/Faces_easy
../content/CaltechDataset/test/sea_horse
../content/CaltechDataset/test/crocodile_head
../content/CaltechDataset/test/garfield
../content/CaltechDataset/test/binocular
../content/CaltechDataset/test/dollar_bill
../content/CaltechDataset/test/panda
../content/CaltechDataset/test/mayfly
../content/CaltechDataset/test/hedgehog
../content/CaltechDataset/test/cannon
../content/CaltechDataset/test/menorah
../content/CaltechDataset/test/platypus
../content/CaltechDataset/test/revolver
../co

In [8]:
i = 0
"""     
for filename in os.listdir(folderpaths[0]): 
       print(filename)


filename = os.listdir(folderpaths[0])[0]
"""     

filename = os.listdir(folderpaths[0])[2]   
    
print(filename)


image_0027.jpg


In [0]:
#Copying over last Y entries of each directory into test directory -SKIP THIS IF USING SEPARATED DATASET
"""
import shutil
for i in range(len(folderpaths)):
  sourcepath = folderpaths[i]
  destpath  =testpaths[i]
  #print(sourcepath)
  #print(destpath)
  y=1
  while (y<11):
      filename = os.listdir(sourcepath)[y]
      
    
    
      print(filename)
      
     
      src = sourcepath +"/"+filename
      dst = destpath +"/"+filename
      
      shutil.copy(src, dst)
      os.remove(src)
      y += 1
      

!zip -r /content/CheckCaltech.zip /content/CaltechDataset/
"""

'\nimport shutil\nfor i in range(len(folderpaths)):\n  sourcepath = folderpaths[i]\n  destpath  =testpaths[i]\n  #print(sourcepath)\n  #print(destpath)\n  y=1\n  while (y<11):\n      filename = os.listdir(sourcepath)[y]\n      \n    \n    \n      print(filename)\n      \n     \n      src = sourcepath +"/"+filename\n      dst = destpath +"/"+filename\n      \n      shutil.copy(src, dst)\n      os.remove(src)\n      y += 1\n      \n\n!zip -r /content/CheckCaltech.zip /content/CaltechDataset/\n'

In [8]:
#Sanity check

print (folderpaths)

['../content/CaltechDataset/Caltech101/schooner', '../content/CaltechDataset/Caltech101/elephant', '../content/CaltechDataset/Caltech101/crocodile', '../content/CaltechDataset/Caltech101/pigeon', '../content/CaltechDataset/Caltech101/kangaroo', '../content/CaltechDataset/Caltech101/pagoda', '../content/CaltechDataset/Caltech101/rhino', '../content/CaltechDataset/Caltech101/euphonium', '../content/CaltechDataset/Caltech101/brain', '../content/CaltechDataset/Caltech101/llama', '../content/CaltechDataset/Caltech101/sunflower', '../content/CaltechDataset/Caltech101/starfish', '../content/CaltechDataset/Caltech101/Faces_easy', '../content/CaltechDataset/Caltech101/sea_horse', '../content/CaltechDataset/Caltech101/crocodile_head', '../content/CaltechDataset/Caltech101/garfield', '../content/CaltechDataset/Caltech101/binocular', '../content/CaltechDataset/Caltech101/dollar_bill', '../content/CaltechDataset/Caltech101/panda', '../content/CaltechDataset/Caltech101/mayfly', '../content/CaltechDa

In [0]:
#Scaling Images from here onwards - FOR SEPARATED DATASET ONLY


"""


import os
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img, save_img
import tensorflow as tf
tf.enable_eager_execution()




percentages=[20,40,60]

#Issue fixed: we introduce a check for both that its an image, and further that it doesnt e
for percent in percentages:
  for foldpath in folderpaths:
    for image in os.listdir(foldpath):
      filename = os.path.join(foldpath,image)
      #print(filename)
      if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        ending =filename[-11: ]
        percentending='percent.jpg'
        if ending != percentending:
          scaledimg = scaleImage(filename,percent)
          #Change percentage if you for scaling
          fileNewName =image[ :-4] +'_'+str(percent)+'percent.jpg'
          filesave=os.path.join(foldpath,fileNewName)
          save_img(filesave, scaledimg)
          #print(filesave)

"""

"\n\n\nimport os\nfrom keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img, save_img\nimport tensorflow as tf\ntf.enable_eager_execution()\n\n\n\n\npercentages=[20,40,60]\n\n#Issue fixed: we introduce a check for both that its an image, and further that it doesnt e\nfor percent in percentages:\n  for foldpath in folderpaths:\n    for image in os.listdir(foldpath):\n      filename = os.path.join(foldpath,image)\n      #print(filename)\n      if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):\n        ending =filename[-11: ]\n        percentending='percent.jpg'\n        if ending != percentending:\n          scaledimg = scaleImage(filename,percent)\n          #Change percentage if you for scaling\n          fileNewName =image[ :-4] +'_'+str(percent)+'percent.jpg'\n          filesave=os.path.join(foldpath,fileNewName)\n          save_img(filesave, scaledimg)\n          #print(filesave)\n\n"

In [9]:
## Modified for doing staggered approach, only produce 1 percentage at a time

import os
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img, save_img
import tensorflow as tf
tf.enable_eager_execution()

percentages =[60]
for percent in percentages:
  for foldpath in folderpaths:
    for image in os.listdir(foldpath):
      filename = os.path.join(foldpath,image)
      print(filename)
      if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        ending =filename[-11: ]
        percentending='percent.jpg'
        if ending != percentending:
          scaledimg = scaleImage(filename,percent)
          #Change percentage if you for scaling
          fileNewName =image[ :-4] +'_'+str(percent)+'percent.jpg'
          filesave=os.path.join(foldpath,fileNewName)
          save_img(filesave, scaledimg)
          os.remove(filename) 
          #print(filesave)
      

#Change filename as appropriate
!zip -r /content/preprocessed_Caltech_60.zip /content/CaltechDataset

Using TensorFlow backend.


Streaming output truncated to the last 5000 lines.
  adding: content/CaltechDataset/Caltech101/trilobite/image_0077_60percent.jpg (deflated 2%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0050_60percent.jpg (deflated 2%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0051_60percent.jpg (deflated 2%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0036_60percent.jpg (deflated 1%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0071_60percent.jpg (deflated 1%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0081_60percent.jpg (deflated 1%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0056_60percent.jpg (deflated 1%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0069_60percent.jpg (deflated 2%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0075_60percent.jpg (deflated 2%)
  adding: content/CaltechDataset/Caltech101/trilobite/image_0058_60percent.jpg (deflated 2%)
  adding: content/C

In [0]:
!zip -r /content/preprocessed_Caltech.zip /content/CaltechDataset


from google.colab import files
files.download("/content/preprocessedPronet.zip")

Streaming output truncated to the last 5000 lines.
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0241_60percent.jpg (deflated 21%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0359_20percent.jpg (deflated 1%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0222_20percent.jpg (deflated 3%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0266_60percent.jpg (deflated 4%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0392.jpg (deflated 1%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0461.jpg (deflated 3%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0027_40percent.jpg (deflated 2%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0084_20percent.jpg (deflated 1%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Google/image_0276_60percent.jpg (deflated 0%)
  adding: content/CaltechDataset/Caltech101/BACKGROUND_Goog

FileNotFoundError: ignored

In [0]:
#Internal TF methods- works 25-12-2019 

#see https://www.wouterbulten.nl/blog/tech/data-augmentation-using-tensorflow-data-dataset/






tfimg = tf.convert_to_tensor(np.asarray(img))
#image = tf.image.resize_images(img, (800, 800))


def preprocess_input(x):
    
    x = tf.image.random_hue(x, 0.2)
    x = tf.image.random_saturation(x, 0.8, 1.4)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.9, 1.1)
    
    return x

tf_img=preprocess_input(tfimg)
to_img = array_to_img(tf_img)
plt.figure()
plt.imshow(tf_img) #Works with either tf or to_img
plt.show()

In [0]:
#IMGAUG https://github.com/aleju/imgaug
#Original code for sequences of images, saved for posterity
"""
def augment_images(images):
    ia.seed(numpy.random.randint(100000, size=1)[0])
    for image in images:
        #image might not have 3 dimensions
        if np.ndim(image) < 3:
            image = image[np.newaxis,:, :]
    # Example batch of images.
    # The array has shape (32, 64, 64, 3) and dtype uint8.

    seq = iaa.Sequential([
            #iaa.Fliplr(0.5), # horizontal flips
        iaa.Crop(percent=(0, 0.1)), # random crops
        # Small gaussian blur with random sigma between 0 and 0.5.
        # But we only blur about 50% of all images.
        iaa.Sometimes(0.5,
            iaa.GaussianBlur(sigma=(0, 0.5))
            ),
        # Strengthen or weaken the contrast in each image.
        iaa.ContrastNormalization((0.75, 1.5)),
        # Add gaussian noise.
        # For 50% of all images, we sample the noise once per pixel.
        # For the other 50% of all images, we sample the noise per pixel AND
        # channel. This can change the color (not only brightness) of the
        # pixels.
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
        # Make some images brighter and some darker.
        # In 20% of all cases, we sample the multiplier once per channel,
        # which can end up changing the color of the images.
        iaa.Multiply((0.8, 1.2), per_channel=0.2),
        # Apply affine transformations to each image.
        # Scale/zoom them, translate/move them, rotate them and shear them.
        iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-25, 25),
            shear=(-8, 8)
            )
        ], random_order=True) # apply augmenters in random order

    images_aug = seq.augment_images(images)
    return images_aug
"""

#The function above also has random crops, very useful for simulating youtube environments.
def augment_image(image):
    ia.seed(numpy.random.randint(100000, size=1)[0])
    
        #image might not have 3 dimensions
    if np.ndim(image) < 3:
        image = image[np.newaxis,:, :]
    # Example batch of images.
    # The array has shape (32, 64, 64, 3) and dtype uint8.

    seq = iaa.Sequential([
            #iaa.Fliplr(0.5), # horizontal flips
        #iaa.Crop(percent=(0, 0.1)), # random crops
        # Small gaussian blur with random sigma between 0 and 0.5.
        # But we only blur about 50% of all images.
        iaa.GaussianBlur(sigma=(0.7, 1)),
        # Strengthen or weaken the contrast in each image.
        iaa.ContrastNormalization((0.75, 1.5)),
        # Add gaussian noise.
        # For 50% of all images, we sample the noise once per pixel.
        # For the other 50% of all images, we sample the noise per pixel AND
        # channel. This can change the color (not only brightness) of the
        # pixels.
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
        # Make some images brighter and some darker.
        # In 20% of all cases, we sample the multiplier once per channel,
        # which can end up changing the color of the images.
        iaa.Multiply((0.8, 1.2), per_channel=0.2),
        # Apply affine transformations to each image.
        # Scale/zoom them, translate/move them, rotate them and shear them.
        
        ], random_order=True) # apply augmenters in random order

    image_aug = seq.augment_image(image)
    return image_aug


def write_image(path,image):
    imageio.mimwrite(path+".tiff",image, format="TIFF")


imgaug_img=augment_image(img)
plt.figure()
plt.imshow(imgaug_img) #Works with either tf or to_img
plt.show()


In [0]:
#OPENCV2 noise

import numpy as np
import random
import cv2

def sp_noise(image,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

#cv2.imwrite('sp_noise.jpg', noise_img)


#Guassian blur
img_cv2 = cv2.GaussianBlur(img, (15,15), 0)
plt.figure()
plt.imshow(img_cv2) #Works with either tf or to_img
plt.show()
